# Pairing

> Functions to do the test pairings and video pairings.

In [ ]:
#| default_exp pairing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti

import math
import pandas as pd
import numpy as np
from typing import Tuple
from itertools import islice

from sqlalchemy.orm import aliased
from collections import defaultdict

from sql_face.tables import *

## Test pairs

In [ ]:
#| export

def get_test_pairs(enfsi_years, session):

       test_pairs = (session.query(EnfsiPair)
                     .filter(EnfsiPair.second.has(EnfsiImage.year.in_(enfsi_years)))
                     .all()
                     )
       return test_pairs

In [ ]:
#| export

def get_valid_test_pairs_2015(session, 
                        detector,
                        embeddingModel,
                        qualityModel,
                        quality_dropouts)->Tuple[EnfsiPair2015, FaceImage, FaceImage, float]:

    
    # todo: alias not needed anymore. Remove them.
    first_cropped_image = aliased(CroppedImage)
    second_cropped_image = aliased(CroppedImage)
    first_detector = aliased(Detector)
    second_detector = aliased(Detector)
    first_face_image = aliased(FaceImage)
    second_face_image = aliased(FaceImage)
    first_emb = aliased(EmbeddingModel)
    second_emb = aliased(EmbeddingModel)
    first_qua = aliased(QualityModel)
    second_qua = aliased(QualityModel)

    # Det&Emb
    det_id = session.query(Detector.detector_id).filter(Detector.name == detector).one()[0]
    emb_id = \
    session.query(EmbeddingModel.embeddingModel_id).filter(EmbeddingModel.name == embeddingModel).one()[0]
    qua_id = \
    session.query(QualityModel.qualityModel_id).filter(QualityModel.name == qualityModel).one()[0]

    
    query_face_img_id = session.query(CroppedImage.image_id, FaceImage, QualityImage.quality) \
        .filter(CroppedImage.detector_id == det_id, CroppedImage.face_detected == True) \
        .filter(FaceImage.croppedImage_id == CroppedImage.croppedImage_id) \
        .filter(FaceImage.embeddingModel_id == emb_id) \
        .filter(QualityImage.faceImage_id == FaceImage.faceImage_id) \
        .filter(QualityImage.qualityModel_id == qua_id).all()

    # a cada image_id asociamos su (FaceImage,  quality)
    face_image_dict = defaultdict(Tuple)
    for row in query_face_img_id:
        face_image_dict[row[0]] = (row[1], row[2]) 

    
    subquery_pair = session.query(EnfsiPair2015.enfsiPair2015_id)\
        .join(first_cropped_image, EnfsiPair2015.first_id == first_cropped_image.image_id) \
        .filter(first_cropped_image.detector_id == det_id,
                first_cropped_image.face_detected == True).subquery()


    query_pair = session.query(EnfsiPair2015) \
        .join(second_cropped_image, EnfsiPair2015.second_id == second_cropped_image.image_id) \
        .filter(second_cropped_image.detector_id == det_id,
                second_cropped_image.face_detected == True,
                EnfsiPair2015.enfsiPair2015_id.in_(subquery_pair))

    
    all_pairs = query_pair.all()

    comparisons = list(set([enfsi_pair.comparison for enfsi_pair in all_pairs]))

    best_pairs = []

    # imagen promedio por comparison, no existe en Database
    df_mean_image = pd.DataFrame(columns = ['Comparison','embedding1','embedding2', 'y'])
    
    for x in comparisons:

        comp_pairs = [(pair, min(face_image_dict[pair.first.image_id][1], face_image_dict[pair.second.image_id][1])) \
            for pair in all_pairs if pair.comparison == x]

        
        comp_pairs.sort(key=lambda x:x[1], reverse=True)
        qua_pairs = [comp_pair[0] for comp_pair in comp_pairs]

        for dropout in quality_dropouts:
            # best_pairs+=comp_pairs[:int(len(comp_pairs)*dropout)]
            truncated_pairs = list(islice(qua_pairs, math.ceil(len(qua_pairs) * dropout)))
            drop_pairs = [(pair, dropout) for pair in truncated_pairs]
            best_pairs += drop_pairs



        emb1  = [face_image_dict[pair[0].first.image_id][0].embeddings for pair in comp_pairs]
        weight1 = [face_image_dict[pair[0].first.image_id][1] for pair in comp_pairs]
        embedding_first = np.dot(weight1,emb1)/np.sum(weight1)

        emb2  = [face_image_dict[pair[0].second.image_id][0].embeddings for pair in comp_pairs]
        weight2 = [face_image_dict[pair[0].second.image_id][1] for pair in comp_pairs]
        embedding_second = np.dot(weight2,emb2)/np.sum(weight2)

        y = comp_pairs[0][0].same

        df_mean_image = df_mean_image.append({'Comparison':x,'embedding1':embedding_first,'embedding2':embedding_second, 'y':y}, ignore_index = True)

        #emb1 ,weigth2 = [face_image_dict[pair.second.image_id][0].embeddings, face_image_dict[pair.second.image_id][1] for pair in comp_pairs]


        

    valid_test_pairs = [(pair[0], face_image_dict[pair[0].first.image_id][0],\
                        face_image_dict[pair[0].second.image_id][0], pair[1],pair[0].get_n_common_attributes())
                        for pair in best_pairs] #(pair, emb1,emb2,quality_drop,n_common_attributes)
    
    return valid_test_pairs, df_mean_image

In [ ]:
#| export

def get_valid_test_pairs(session, 
                        detector,
                        embeddingModel,
                        enfsi_years) -> Tuple[EnfsiPair, FaceImage, FaceImage]:

    first_cropped_image = aliased(CroppedImage)
    second_cropped_image = aliased(CroppedImage)
    first_detector = aliased(Detector)
    second_detector = aliased(Detector)
    first_face_image = aliased(FaceImage)
    second_face_image = aliased(FaceImage)
    first_emb = aliased(EmbeddingModel)
    second_emb = aliased(EmbeddingModel)

    # Det&Emb
    det_id = session.query(Detector.detector_id).filter(Detector.name == detector).one()[0]
    emb_id = \
    session.query(EmbeddingModel.embeddingModel_id).filter(EmbeddingModel.name == embeddingModel).one()[0]

    query_face_img_id = session.query(CroppedImage.image_id, FaceImage) \
        .filter(CroppedImage.detector_id == det_id, CroppedImage.face_detected == True) \
        .filter(FaceImage.croppedImage_id == CroppedImage.croppedImage_id) \
        .filter(FaceImage.embeddingModel_id == emb_id).all()

    face_image_dict = defaultdict(FaceImage)
    for row in query_face_img_id:
        face_image_dict[row[0]] = row[1]

    query_pair_id = session.query(EnfsiPair).filter(EnfsiPair.second.has(EnfsiImage.year.in_(enfsi_years)))

    query_1 = query_pair_id \
        .join(first_cropped_image, EnfsiPair.first_id == first_cropped_image.image_id) \
        .filter(first_cropped_image.detector_id == det_id,
                first_cropped_image.face_detected == True)

    query_2 = query_pair_id \
        .join(second_cropped_image, EnfsiPair.second_id == second_cropped_image.image_id) \
        .filter(second_cropped_image.detector_id == det_id,
                second_cropped_image.face_detected == True)

    query = (query_1.intersect(query_2).all())

    # The one at the end indicates all the pairs are chosen (see year 2015).
    valid_test_pairs = [(pair, face_image_dict[pair.first.image_id],\
                        face_image_dict[pair.second.image_id], 1, pair.get_n_common_attributes())
                        for pair in query]
    
    return valid_test_pairs

In [ ]:
#| export

def get_test_pairs_per_category(session,
                                #new
                                #image_filters,
                                #face_image_filters,
                                filters,

                                detector,
                                embeddingModel,
                                qualityModel,
                                enfsi_years,
                                quality_dropout):

    valid_test_pairs = []
    df_pairs_2015 = None
    
    enfsi_short = enfsi_years.copy()
    
    if 2015 in enfsi_years:

        enfsi_short.remove(2015)
        valid_test_pairs, df_pairs_2015 = get_valid_test_pairs_2015(session,
                                            detector,
                                            embeddingModel,
                                            qualityModel,
                                            quality_dropout)
        
        
    valid_test_pairs += get_valid_test_pairs(session,
                                            detector,
                                            embeddingModel,
                                            #qualityModel,
                                            enfsi_short) 
    #quitado para 2015
    

    test_categories = [
        row[0].get_category(session, filters, detector, embeddingModel, qualityModel)
        #row[0].get_category(image_filters, face_image_filters, detector, embeddingModel)
        for row in valid_test_pairs]

    test_pairs_per_category = defaultdict(list)

    for row_enfsi_pair, category in zip(valid_test_pairs, test_categories):
        test_pairs_per_category[category].append(row_enfsi_pair)
    return test_pairs_per_category, df_pairs_2015

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()